In [208]:
import pandas as pd
from datetime import datetime
import re

In [275]:
df = pd.read_csv('../data/absorbance/raw.csv')

# extracting the year, so that we can remove the wrong data

In [276]:
regex = '^\w+.\w+,\d{2}/\d{2}/(\d{4})$'

In [277]:
df['year'] = df[df.columns.values[0]].apply(lambda x: re.findall(regex, x)[0])

# dropping data with wrong date

In [278]:
rows_to_drop = df[df['year'] == '1601']
df.drop(rows_to_drop.index, inplace=True)

In [279]:
df.reset_index(drop=True, inplace=True)

# getting shelf life and treatment info

In [280]:
regex = '^\w*(T\d)(B\d)(\d)'

In [281]:
df['TREATMENT'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][0])
df['BLOCK'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][1])
df['SHELF-LIFE'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][2])

# removing unnecessary info

In [282]:
cols = list(df.columns.values[:2])
cols.append('year')

In [283]:
df.drop(cols, axis=1, inplace=True)

# saving new data

In [284]:
df.to_csv('../data/absorbance/treated.csv')

# removing wrong felix data

In [285]:
# Removing these bc in the felix data, there are 20 samples for B1 and 24 for B3, whereas in the quality .csv
# there are 42 samples in total. Assuming that the 24 B3 samples are correct (since this should be the exact
# value), there should be only 18 samples for B1. Thus, I'm removing the last two from B1
df[df['TREATMENT'] == 'T6'][df['SHELF-LIFE'] == '1'].iloc[18:20]

/home/danielesreis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,285,288,291,294,297,300,303,306,309,312,...,1182,1185,1188,1191,1194,1197,1200,TREATMENT,BLOCK,SHELF-LIFE
660,0,0,0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,T6,B1,1
661,0,0,0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,T6,B1,1


In [286]:
df.drop([660, 661], inplace=True)

In [287]:
df.to_csv('../data/absorbance/treated_wo_incorrect.csv')

# appending the quality data

In [288]:
qlt = pd.read_csv('../data/quality/raw.csv')

In [289]:
qlt.drop(columns=['LEVELS'], inplace=True)
qlt['TREATMENT'] = qlt['TREATMENT'].apply(lambda x: str(x))
qlt['SHELF-LIFE'] = qlt['SHELF-LIFE'].apply(lambda x: str(x))

# removing wrong quality data

In [290]:
# removing from quality data bc there are 46 samples from felix and 48 from quality .csv. B3 is supposedly correct,
# because it contains 24 felix samples, whereas there's 22 B1 samples. So, I'm gonna remove the last two from B1.
qlt[qlt['TREATMENT']=='T4'][qlt['SHELF-LIFE']=='1'].iloc[22:24]

/home/danielesreis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
492,1,T4,23,513.28,65.47,52.12,62.20,6.9,59.83,64.26,79.27,16.5,NaN,20.51
493,1,T4,24,513.28,71.87,56.41,78.29,13.8,66.32,67.88,79.67,16.2,NaN,19.98


In [291]:
qlt.drop([492, 493], inplace=True)

In [292]:
qlt.to_csv('../data/quality/treated_wo_incorrect.csv')

# indexing by treatment and shelf-life to allow assignment

In [293]:
new_df = df.sort_values(by=['TREATMENT', 'SHELF-LIFE']).reset_index()

In [294]:
new_qlt = qlt.sort_values(by=['TREATMENT', 'SHELF-LIFE']).reset_index()

In [295]:
attrs = list(new_qlt.columns[3:].values)
for att in attrs:
    new_df[att] = new_qlt[att]

In [296]:
cols = ['TREATMENT', 'BLOCK', 'SHELF-LIFE'] + attrs + list(new_df.columns[1:307].values)

In [297]:
cols.remove('NUM')

In [298]:
new_df = new_df[cols]

In [299]:
new_df.set_index('TREATMENT', inplace=True)

In [300]:
new_df.to_csv('../data/data_final.csv')